# Kaggle API Stuff

This shows a little how to use the kaggle api in a notebook. The 'kaggle' command line is good for shell scripts, but I wanted to load data directly into Pandas dataframes and do some analysis. This assumes you have already installed and configured the api via 'pip install kaggle'  

[Kaggle api](https://github.com/Kaggle/kaggle-api) from GitHub.


In [355]:
from kaggle import api
from datetime import datetime

In [356]:
api.authenticate()

In [108]:
rows = []
page=1
c = api.competitions_list(page=1,search='')
while len(c) > 0:
    rows.extend(c)
    page += 1
    c = api.competitions_list(page=page,search='')

In [357]:
print(rows[0].keys())

dict_keys(['ref', 'tags', 'description', 'id', 'title', 'url', 'deadline', 'category', 'reward', 'organizationName', 'organizationRef', 'kernelCount', 'teamCount', 'userHasEntered', 'userRank', 'mergerDeadline', 'newEntrantDeadline', 'enabledDate', 'maxDailySubmissions', 'maxTeamSize', 'evaluationMetric', 'awardsPoints', 'isKernelsSubmissionsOnly', 'submissionsDisabled'])


In [358]:
cols = ['ref',
        'enabledDate',
        'deadline',
        'organizationName',
        'organizationRef',
        'reward',
        'title',
        'description',
        'teamCount',
        'url']

In [369]:
df = pd.DataFrame.from_dict(rows)[cols]

## Cleanup - column types and values

In [387]:
from re import sub
import locale

def reward_value(s):
    locale.setlocale(locale.LC_ALL,'')
    sv = sub('\$','',s)
    sv = sub('€','',sv)
    try:
        return locale.atof(sv)
    except ValueError:
        return 0.0

In [388]:
dfc = pd.concat([
    pd.to_datetime(df.enabledDate),
    df.ref,
    pd.to_datetime(df.deadline),
    df.organizationName,
    df.organizationRef.fillna('NA'),
    df.reward.apply(reward_value),
    df.title,
    df.description,
    pd.to_numeric(df.teamCount),
    df.url
],axis=1)

In [389]:
# Optional:
# this attemps to fill null organization name with first two words of title - as it seems
# to have a company name many times.

dfc['organizationName'].fillna(dfc['title'].apply(lambda x: ' '.join(x.split()[:2])),inplace=True)

In [390]:
dfc.set_index(pd.DatetimeIndex(dfc['enabledDate']),inplace=True,drop=True)
dfc.drop(dfc.columns[0],inplace=True,axis=1)

In [391]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 277 entries, 2017-07-26 15:08:10.333000 to 2010-04-07 07:57:43
Data columns (total 9 columns):
ref                 277 non-null object
deadline            277 non-null datetime64[ns]
organizationName    277 non-null object
organizationRef     277 non-null object
reward              277 non-null float64
title               277 non-null object
description         277 non-null object
teamCount           277 non-null int64
url                 277 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 21.6+ KB


In [392]:
dfc.to_csv('kaggle_contests.csv')

## Query current contests that have a monetary value

In [393]:
current_reward_contests = dfc[(dfc.deadline > datetime.today()) & (dfc.reward > 0)]
current_reward_contests

,ref,deadline,organizationName,organizationRef,reward,title,description,teamCount,url
enabledDate,,,,,,,,,
2018-03-07 18:20:47.570,imaterialist-challenge-furniture-2018,2018-05-30 23:59:00,iMaterialist Challenge,NA,2500.0,iMaterialist Challenge (Furniture) at FGVC5,Image Classification of Furniture & Home Goods.,45,https://www.kaggle.com/c/imaterialist-challeng...
2018-02-02 23:01:00.063,landmark-retrieval-challenge,2018-05-22 23:59:00,Google,google,2500.0,Google Landmark Retrieval Challenge,"Given an image, can you find all of the same l...",78,https://www.kaggle.com/c/landmark-retrieval-ch...
2018-02-02 23:00:50.797,landmark-recognition-challenge,2018-05-22 23:59:00,Google,google,2500.0,Google Landmark Recognition Challenge,Label famous (and not-so-famous) landmarks in ...,143,https://www.kaggle.com/c/landmark-recognition-...
2018-03-05 20:05:11.213,talkingdata-adtracking-fraud-detection,2018-05-07 23:59:00,TalkingData,talkingdata,25000.0,TalkingData AdTracking Fraud Detection Challenge,Can you detect fraudulent click traffic for mo...,938,https://www.kaggle.com/c/talkingdata-adtrackin...
2018-01-16 14:00:00.790,data-science-bowl-2018,2018-04-16 23:59:00,Booz Allen Hamilton,BoozAllenHamilton,100000.0,2018 Data Science Bowl,Find the nuclei in divergent images to advance...,2558,https://www.kaggle.com/c/data-science-bowl-2018
2018-02-20 18:54:34.967,mens-machine-learning-competition-2018,2018-04-02 23:59:00,Google Cloud,google-cloud,50000.0,Google Cloud & NCAA® ML Competition 2018-Men's,Apply Machine Learning to NCAA® March Madness®,934,https://www.kaggle.com/c/mens-machine-learning...
2018-02-20 18:54:35.837,womens-machine-learning-competition-2018,2018-04-01 23:59:00,Google Cloud,google-cloud,50000.0,Google Cloud & NCAA® ML Competition 2018-Women's,Apply machine learning to NCAA® March Madness®,505,https://www.kaggle.com/c/womens-machine-learni...
2017-12-19 15:38:59.253,jigsaw-toxic-comment-classification-challenge,2018-03-20 23:59:00,Jigsaw,jigsaw-team,35000.0,Toxic Comment Classification Challenge,Identify and classify toxic online comments,4460,https://www.kaggle.com/c/jigsaw-toxic-comment-...


### Most Popular Contest Ever

In [408]:
dfc.loc[dfc.teamCount.idxmax()]

ref                                                           titanic
deadline                                          2020-04-07 00:00:00
organizationName                                               Kaggle
organizationRef                                                kaggle
reward                                                              0
title                         Titanic: Machine Learning from Disaster
description         Start here! Predict survival on the Titanic an...
teamCount                                                       10257
url                                  https://www.kaggle.com/c/titanic
Name: 2012-09-28 21:13:33.550000, dtype: object

### Most Popular Active Contest

In [411]:
current_contests = dfc[(dfc.deadline > datetime.today()) & (dfc.reward > 0)]

In [415]:
current_contests.loc[current_contests.teamCount.idxmax()]

ref                     jigsaw-toxic-comment-classification-challenge
deadline                                          2018-03-20 23:59:00
organizationName                                               Jigsaw
organizationRef                                           jigsaw-team
reward                                                          35000
title                          Toxic Comment Classification Challenge
description               Identify and classify toxic online comments
teamCount                                                        4460
url                 https://www.kaggle.com/c/jigsaw-toxic-comment-...
Name: 2017-12-19 15:38:59.253000, dtype: object

## Datasets

In [ ]:
rows = []
page=1
c = api.datasets_list(page=1,search='')
while len(c) > 0:
    rows.extend(c)
    page += 1
    c = api.datasets_list(page=page,search='')

In [417]:
print(rows[0].keys())

dict_keys(['ref', 'tags', 'creatorName', 'creatorUrl', 'totalBytes', 'url', 'lastUpdated', 'downloadCount', 'isPrivate', 'isReviewed', 'isPublished', 'isFeatured', 'licenseName', 'description', 'ownerName', 'ownerRef', 'kernelCount', 'title', 'topicCount', 'viewCount', 'voteCount', 'currentVersionNumber'])


In [420]:
cols = ['ref',
        'title',
        'description',
        'creatorName',
        'ownerName',
        'isFeatured',
        'isPublished',
        'voteCount',
        'downloadCount',
        'viewCount',
        'lastUpdated',
        'licenseName',
        'url']

In [421]:
df = pd.DataFrame.from_dict(rows)[cols]

In [423]:
dfd = pd.concat([
    df.ref,
    df.title,
    df.description,
    df.creatorName,
    df.ownerName,
    df.isFeatured,
    df.isPublished,
    pd.to_numeric(df.voteCount),
    pd.to_numeric(df.downloadCount),
    pd.to_numeric(df.viewCount),
    pd.to_datetime(df.lastUpdated),
    df.licenseName,
    df.url
],axis=1)

In [424]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2660 entries, 0 to 2659
Data columns (total 13 columns):
ref              2660 non-null object
title            2660 non-null object
description      2660 non-null object
creatorName      2660 non-null object
ownerName        2660 non-null object
isFeatured       2660 non-null bool
isPublished      2660 non-null bool
voteCount        2660 non-null int64
downloadCount    2660 non-null int64
viewCount        2660 non-null int64
lastUpdated      2660 non-null datetime64[ns]
licenseName      2660 non-null object
url              2660 non-null object
dtypes: bool(2), datetime64[ns](1), int64(3), object(7)
memory usage: 233.9+ KB


### Most popular ever dataset

In [426]:
dfd.loc[dfd.voteCount.idxmax()]

ref                                        mlg-ulb/creditcardfraud
title                                  Credit Card Fraud Detection
description      Anonymized credit card transactions labeled as...
creatorName                                                 Andrea
ownerName                             Machine Learning Group - ULB
isFeatured                                                   False
isPublished                                                   True
voteCount                                                     1296
downloadCount                                                55011
viewCount                                                   453392
lastUpdated                             2016-11-05 09:08:46.503000
licenseName      Database: Open Database, Contents: Database Co...
url                 https://www.kaggle.com/mlg-ulb/creditcardfraud
Name: 460, dtype: object

### Most Prolific Upload Owners

In [459]:
sown = dfd.ownerName.value_counts().reset_index()

In [461]:
sown.columns = ['ownerName','ownerUploads']

In [469]:
dfd = dfd.merge(sown,on='ownerName')

In [472]:
dfd.licenseName.value_counts()

CC0: Public Domain                                       1181
Unknown                                                   487
Other (specified in description)                          295
Data files © Original Authors                             166
CC BY-NC-SA 4.0                                           162
Database: Open Database, Contents: Database Contents      126
CC BY-SA 4.0                                              122
GPL 2                                                      50
Database: Open Database, Contents: © Original Authors      30
CC BY-SA 3.0                                               29
World Bank Dataset Terms of Use                             9
Reddit API Terms                                            3
Name: licenseName, dtype: int64

In [482]:
prolific_uploaders =dfd[dfd.ownerUploads > dfd.ownerUploads.mean() + dfd.ownerUploads.std()]

In [484]:
prolific_uploaders.ownerName.value_counts()

Rachael Tatman          78
UCI Machine Learning    33
Jacob Boysen            29
Sohier Dane             24
PyTorch                 21
Abdullah Karimi         21
PromptCloud             21
Chris Crawford          21
Name: ownerName, dtype: int64